In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
# from utils.vis import plot_dataset_gyration_plane, build_jPCA
from utils.datasets import SyntheticTravelingWave

from utils.utils import (
    subtract_cc_mean,
    soft_normalize,
    compute_lambdas)

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
from jPCA.util import plot_projections, preprocess
import jPCA

from utils.color_palettes import (
    jpca_palette_journ1,
    blue_yellow_cmap,
    cmaps
    )

In [2]:
def peth_normalize(x):
    mean = x.mean(axis=0)
    x_ = x - mean
    pmax = np.abs(x_).max(axis=0)
    pmax = np.where(pmax==0, 1, pmax)
    return x_ / (pmax)


def plot_peth(datas_list, go_cue, ax, fs, sub_mean=False):
    """
    Plot PETHs averaged across conditions for a given data
    """
    datas = soft_normalize(datas_list)
    if sub_mean:
        datas = subtract_cc_mean(datas)

    # sort by average wave maximum
    cc_mean = np.mean(datas, axis=0)
    idxs = np.argmax(cc_mean, axis=0)
    cc_mean_sorted = cc_mean[:, np.argsort(idxs)]
    divider = make_axes_locatable(ax)

    # avg condition
    mean_data_sort_norm = peth_normalize(cc_mean_sorted)
    im1 = ax.imshow(mean_data_sort_norm.T, aspect='auto',
                       cmap=blue_yellow_cmap(), interpolation='none')
    ax.axvline(go_cue, c='black', ls='--', alpha=0.4)
    cax1 = divider.append_axes('right', size='5%', pad=0.05)
    cbar = plt.colorbar(im1, cax=cax1, orientation='vertical')
    cbar.ax.tick_params(labelsize=8)

    tick_locator = ticker.MaxNLocator(nbins=4)
    cbar.locator = tick_locator
    cbar.update_ticks()
    ax.yaxis.set_major_locator(ticker.MultipleLocator(20))


def build_jPCA(data, ax, ts, te, sub_mean=True,
               c_siz=0.001, arr_siz=0.001, cmap=jpca_palette_journ1(),
               fitting=False):
    cond_num = data.shape[0]
    datas_list = [data[i] for i in range(cond_num)]
    times = list(np.arange(datas_list[0].shape[0]))

    data_list = list(soft_normalize(datas_list))
    jpca = jPCA.JPCA(num_jpcs=6)
    (projected, _, _, _) = jpca.fit(data_list, times=times, 
                tstart=ts, tend=te, subtract_cc_mean=sub_mean,
                num_pcs=6, pca=True, soft_normalize=0)
    if fitting:
        projected = [-1.*proj for proj in projected]
    plot_projections(projected, axis=ax, x_idx=0, y_idx=1, circle_size=c_siz, arrow_size=arr_siz,
                     colormap=cmap)

In [8]:
from utils.color_palettes import journ_color_dict, RB1
from utils.utils import generate_response
import cv2
import os
import shutil                                       
import imageio
from tqdm import tqdm

def generate_image_name(idx, gif_dir):
    l = len(str(idx))
    if l == 1:
        n = '00' + str(idx)
    elif l == 2:
        n = '0' + str(idx)
    elif l == 3:
        n = str(idx)
    return gif_dir + n + '.png'


def render_gif(image_folder, video_name):
    # image_folder = '/home/kate/Documents/brainlearning/imgs/tt/'
    # video_name = '/home/kate/Documents/brainlearning/imgs/' + 'wavelet400.gif'

    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    keys = [int(image[:-4]) for image in images]
    xx = np.argsort(keys)
    images = np.array(images)[xx]

    with imageio.get_writer(video_name, mode='I') as writer:
        for filename in images:
            image = imageio.imread(image_folder + "/" + filename)
            writer.append_data(image)


params = [
        #noise free
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':100, 'ncond':8, 'wave_speed':5.1, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':180, 'ncond':8, 'wave_speed':8.1, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':2.0, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':1.1, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':0.9, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':0.8, 'color':journ_color_dict['green']},
        {'amp_noise':0, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':0.3, 'color':journ_color_dict['green']},
        #noised
        {'amp_noise':0.12, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':2.1, 'color':journ_color_dict['blue']},
        {'amp_noise':0., 'phase_noise':0, 'sigma_noise':400, 'sigma':200, 'ncond':8, 'wave_speed':2.1, 'color':journ_color_dict['blue']},  
        {'amp_noise':0.3, 'phase_noise':0, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':2.1, 'color':journ_color_dict['blue']},
        {'amp_noise':0., 'phase_noise':60, 'sigma_noise':0, 'sigma':200, 'ncond':8, 'wave_speed':2.1, 'color':journ_color_dict['blue']},
        {'amp_noise':0.3, 'phase_noise':60, 'sigma_noise':400, 'sigma':200, 'ncond':8, 'wave_speed':2.1, 'color':journ_color_dict['blue']},
        {'amp_noise':0.3, 'phase_noise':60, 'sigma_noise':400, 'sigma':200, 'ncond':8, 'wave_speed':0.6, 'color':journ_color_dict['blue']},
        {'amp_noise':0.05, 'phase_noise':10, 'sigma_noise':1, 'sigma':200, 'ncond':8, 'wave_speed':1.1, 'color':journ_color_dict['blue']},
        {'amp_noise':0.05, 'phase_noise':10, 'sigma_noise':1, 'sigma':200, 'ncond':8, 'wave_speed':0.8, 'color':journ_color_dict['blue']},
        {'amp_noise':0.1, 'phase_noise':20, 'sigma_noise':10, 'sigma':200, 'ncond':8, 'wave_speed':0.8, 'color':journ_color_dict['blue']},
        {'amp_noise':0.1, 'phase_noise':20, 'sigma_noise':10, 'sigma':200, 'ncond':8, 'wave_speed':0.7, 'color':journ_color_dict['blue']},
        {'amp_noise':0.1, 'phase_noise':20, 'sigma_noise':10, 'sigma':200, 'ncond':8, 'wave_speed':0.3, 'color':journ_color_dict['blue']},
          ]
fsize = 15

b_i = 3.2
eps_amp = 0
eps_sigma = 0
eps_phase = 0
sig = 200

# for sig in np.arange(200, 10, -10):
ii = 0
for b_i in tqdm(np.arange(0.3, 3.2, 0.025)):
        p = params[0]
        fig = plt.figure(figsize=(15, 6), constrained_layout=True)
        gs = GridSpec(1, 10, figure=fig) 
        ax0 = fig.add_subplot(gs[:, :4])
        ax1 = fig.add_subplot(gs[:, 4:8])
        ax2 = fig.add_subplot(gs[:, 8:])

        datas_list = [generate_response(t_max=600,
                                        N=200, 
                                        sigma=sig, #p['sigma'], 
                                        a=0,
                                        b=b_i, #p['wave_speed'],
                                        steps=200, 
                                        amp_mean=ampi,
                                        sigma_noise=eps_sigma, #p['sigma_noise'],
                                        phase_noise=eps_phase, #p['phase_noise'],
                                        amp_noise=eps_amp,
                                        )[0].T 
                                for ampi in np.linspace(0.3, 1.2, p['ncond'])] 
        
        times = np.arange(datas_list[0].shape[0]).tolist()
        start, end = times[0], times[-1]
        datas, data_list, full_data_var, pca_variance_captured, pca = preprocess(
                                                                datas_list, 
                                                                times=times, 
                                                                tstart=start,
                                                                tend=end,
                                                                soft_normalize=5, 
                                                                subtract_cc_mean=True, 
                                                                pca=False,
                                                                extra=True)
        jpca = jPCA.JPCA(num_jpcs=6)
        (projected, _, _, _) = jpca.fit(datas_list, times=times, 
                tstart=start, tend=end, subtract_cc_mean=True, num_pcs=6, pca=True)
        
        plot_projections(projected, axis=ax1, x_idx=0, y_idx=1,
                         circle_size=0.000, arrow_size=0.000,
                         colormap=RB1()) 
        ax1.set_xticks([])
        ax1.set_yticks([])
        #==========================================
        plot_peth(datas_list, go_cue=0, ax=ax2, fs=0.001, sub_mean=False)
        ax2.set_title('PETH', fontsize=fsize)
        ax0.set_title('Gyration Plane', fontsize=fsize)
        ax1.set_title('jPCA Plane', fontsize=fsize)
        #==========================================

        Xeig = compute_lambdas(datas)
        xnom = np.abs(np.real(Xeig[0])) + np.abs(np.real(Xeig[1]))
        ynom = np.abs(np.imag(Xeig[0])) + np.abs(np.imag(Xeig[1]))
        x = xnom / (np.abs(Xeig).sum())
        y = ynom / (np.abs(Xeig).sum())
        
        # Params of Gyration Plane
        ins_size = 0.21
        ax=ax0
        ax.set_xlim([-0.05, 1.05])
        ax.set_ylim([-0.05, 1.05])
        ax.plot([0,0], [0,1], lw=1, ls='--', color='grey')
        ax.plot([0,1], [0,0], lw=1, ls='--', color='grey')
        ax.plot([0,1], [1,1], lw=1, ls='--', color='grey')
        ax.plot([1,1], [0,1], lw=1, ls='--', color='grey')

        ax.plot([0,1], [0,1], lw=1, ls='--', color='grey', alpha=0.4)
        circle2 = plt.Circle((0, 0), 1, color='grey',lw=1,
                             ls='--', fill=False, alpha=0.4)
        ax.add_patch(circle2)

        ax.grid(alpha=0.3)
        ax.set_ylabel('Rotation axis', fontsize=12)
        ax.set_xlabel('Decay axis', fontsize=12)
        ax.set_facecolor('#fafafa')
        ax.tick_params(axis='both', which='major', labelsize=9, direction="in")

        ins = ax.inset_axes([x - ins_size/2, y - ins_size/2,
                             ins_size, ins_size], transform=ax.transData)
        ins.set_aspect('equal', adjustable='datalim')
        plot_projections(projected, axis=ins, x_idx=0, y_idx=1,
                         circle_size=0.000, arrow_size=0.000,
                         colormap=RB1()) 
        if x > 0.7:
                hha = 'right'
                title_x = -0.05
                title_y = 0.0
        else:
                hha = 'left'
                title_x = 1.05
                title_y = 0.0
        ins.set_title(  r'$\epsilon_{A}$='+f'{p["amp_noise"]} \n' + 
                        r'$\epsilon_{\sigma}$='+f'{p["sigma_noise"]} \n' + 
                        r'$\epsilon_{k}$='+f'{p["phase_noise"]} \n' + 
                        r'$b$='+f'{round(b_i, 2)} \n' + 
                        r'$\sigma$='+f'{round(sig, 2)}', 
                        fontsize=8, x=title_x, y=title_y, ha=hha)
        ins.set_xticks([])
        ins.set_yticks([])
        for spine in ins.spines.values():
                spine.set_linewidth(2.1)

        nam = generate_image_name(ii, 'image/gif_for_readme/')
        plt.savefig(nam, dpi=90, format='png', bbox_inches='tight', pad_inches=0.0)
        # plt.tight_layout()
        # plt.show()
        plt.clf() 
        ii += 1
#=======================================================================
#=======================================================================
#=======================================================================
#=======================================================================
ii = 117
b_i = 3.2
eps_amp = 0
eps_sigma = 0
eps_phase = 0
sig = 200
for sig in tqdm(np.arange(200, 10, -2)):
        p = params[0]
        fig = plt.figure(figsize=(15, 6), constrained_layout=True)
        gs = GridSpec(1, 10, figure=fig) 
        ax0 = fig.add_subplot(gs[:, :4])
        ax1 = fig.add_subplot(gs[:, 4:8])
        ax2 = fig.add_subplot(gs[:, 8:])

        datas_list = [generate_response(t_max=600,
                                        N=200, 
                                        sigma=sig, #p['sigma'], 
                                        a=0,
                                        b=b_i, #p['wave_speed'],
                                        steps=200, 
                                        amp_mean=ampi,
                                        sigma_noise=eps_sigma, #p['sigma_noise'],
                                        phase_noise=eps_phase, #p['phase_noise'],
                                        amp_noise=eps_amp,
                                        )[0].T 
                                for ampi in np.linspace(0.3, 1.2, p['ncond'])] 
        
        times = np.arange(datas_list[0].shape[0]).tolist()
        start, end = times[0], times[-1]
        datas, data_list, full_data_var, pca_variance_captured, pca = preprocess(
                                                                datas_list, 
                                                                times=times, 
                                                                tstart=start,
                                                                tend=end,
                                                                soft_normalize=5, 
                                                                subtract_cc_mean=True, 
                                                                pca=False,
                                                                extra=True)
        jpca = jPCA.JPCA(num_jpcs=6)
        (projected, _, _, _) = jpca.fit(datas_list, times=times, 
                tstart=start, tend=end, subtract_cc_mean=True, num_pcs=6, pca=True)
        
        plot_projections(projected, axis=ax1, x_idx=0, y_idx=1,
                         circle_size=0.000, arrow_size=0.000,
                         colormap=RB1()) 
        ax1.set_xticks([])
        ax1.set_yticks([])
        #==========================================
        plot_peth(datas_list, go_cue=0, ax=ax2, fs=0.001, sub_mean=False)
        ax2.set_title('PETH', fontsize=fsize)
        ax0.set_title('Gyration Plane', fontsize=fsize)
        ax1.set_title('jPCA Plane', fontsize=fsize)
        #==========================================

        Xeig = compute_lambdas(datas)
        xnom = np.abs(np.real(Xeig[0])) + np.abs(np.real(Xeig[1]))
        ynom = np.abs(np.imag(Xeig[0])) + np.abs(np.imag(Xeig[1]))
        x = xnom / (np.abs(Xeig).sum())
        y = ynom / (np.abs(Xeig).sum())
        
        # Params of Gyration Plane
        ins_size = 0.21
        ax=ax0
        ax.set_xlim([-0.05, 1.05])
        ax.set_ylim([-0.05, 1.05])
        ax.plot([0,0], [0,1], lw=1, ls='--', color='grey')
        ax.plot([0,1], [0,0], lw=1, ls='--', color='grey')
        ax.plot([0,1], [1,1], lw=1, ls='--', color='grey')
        ax.plot([1,1], [0,1], lw=1, ls='--', color='grey')

        ax.plot([0,1], [0,1], lw=1, ls='--', color='grey', alpha=0.4)
        circle2 = plt.Circle((0, 0), 1, color='grey',lw=1,
                             ls='--', fill=False, alpha=0.4)
        ax.add_patch(circle2)

        ax.grid(alpha=0.3)
        ax.set_ylabel('Rotation axis', fontsize=12)
        ax.set_xlabel('Decay axis', fontsize=12)
        ax.set_facecolor('#fafafa')
        ax.tick_params(axis='both', which='major', labelsize=9, direction="in")

        ins = ax.inset_axes([x - ins_size/2, y - ins_size/2,
                             ins_size, ins_size], transform=ax.transData)
        ins.set_aspect('equal', adjustable='datalim')
        plot_projections(projected, axis=ins, x_idx=0, y_idx=1,
                         circle_size=0.000, arrow_size=0.000,
                         colormap=RB1()) 
        if x > 0.7:
                hha = 'right'
                title_x = -0.05
                title_y = 0.0
        else:
                hha = 'left'
                title_x = 1.05
                title_y = 0.0
        ins.set_title(  r'$\epsilon_{A}$='+f'{p["amp_noise"]} \n' + 
                        r'$\epsilon_{\sigma}$='+f'{p["sigma_noise"]} \n' + 
                        r'$\epsilon_{k}$='+f'{p["phase_noise"]} \n' + 
                        r'$b$='+f'{round(b_i, 2)} \n' + 
                        r'$\sigma$='+f'{round(sig, 2)}', 
                        fontsize=8, x=title_x, y=title_y, ha=hha)
        ins.set_xticks([])
        ins.set_yticks([])
        for spine in ins.spines.values():
                spine.set_linewidth(2.1)

        nam = generate_image_name(ii, 'image/gif_for_readme/')
        plt.savefig(nam, dpi=90, format='png', bbox_inches='tight', pad_inches=0.0)
        plt.clf();          
        ii += 1
        # break

  0%|          | 0/117 [00:00<?, ?it/s]

100%|██████████| 95/95 [01:12<00:00,  1.31it/s]


<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

...

<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

<Figure size 1080x432 with 0 Axes>

In [9]:
render_gif(image_folder='image/gif_for_readme/', video_name='gif_for_README_90dpi.gif')

/home/kate/anaconda3/envs/CN/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


In [ ]:
# preprocess_data(self, sigma, a, b, ncond, N, t_max, steps,
#                         sigma_noise=0.0, phase_noise=0.0, amp_noise=0.0,
#                         amp_range=[0.5, 1.0])